In [0]:
import os
import torch
import torchvision
import numpy as np
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [0]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [0]:
# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5,),
                                     std=(0.5,))])

In [5]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 27596664.21it/s]                            


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


32768it [00:00, 452417.32it/s]
  1%|          | 16384/1648877 [00:00<00:11, 142524.22it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


1654784it [00:00, 6969562.96it/s]                           
8192it [00:00, 176805.83it/s]


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw
Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


In [0]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [0]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1))

In [0]:
# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [0]:
# Device setting
D = D.to(device)
G = G.to(device)

In [0]:
# Optimizers
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [0]:
# Loss weight for gradient penalty
lambda_gp = 10


def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = torch.Tensor(np.random.random((real_samples.size(0),1))).to(device)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = torch.autograd.Variable(torch.Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False).to(device)
    # Get gradient w.r.t. interpolates
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [12]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #
        for di in range(5):
          z = torch.randn(batch_size, latent_size).to(device)
          fake_images = G(z)
          D_real = D(images)
          D_fake = D(fake_images)
          
          # Gradient penalty
          gradient_penalty = compute_gradient_penalty(D, images.data, fake_images.data)
        
          # Backprop and optimize
          d_loss = -(torch.mean(D_real) - torch.mean(D_fake)) + lambda_gp * gradient_penalty
          
          reset_grad()
          d_loss.backward()
          d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        D_fake = D(fake_images)
        
        g_loss = -torch.mean(D_fake)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          D_real.mean().item(), D_fake.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/200], Step [200/600], d_loss: -6.2720, g_loss: -1.4612, D(x): 8.82, D(G(z)): 1.46
Epoch [0/200], Step [400/600], d_loss: -7.3625, g_loss: -2.0917, D(x): 10.75, D(G(z)): 2.09
Epoch [0/200], Step [600/600], d_loss: -5.9951, g_loss: -2.8941, D(x): 9.29, D(G(z)): 2.89
Epoch [1/200], Step [200/600], d_loss: -5.8316, g_loss: -1.4316, D(x): 7.80, D(G(z)): 1.43
Epoch [1/200], Step [400/600], d_loss: -5.5035, g_loss: 1.2840, D(x): 4.75, D(G(z)): -1.28
Epoch [1/200], Step [600/600], d_loss: -5.1433, g_loss: 2.5775, D(x): 3.33, D(G(z)): -2.58
Epoch [2/200], Step [200/600], d_loss: -5.9873, g_loss: 3.4824, D(x): 3.31, D(G(z)): -3.48
Epoch [2/200], Step [400/600], d_loss: -5.5139, g_loss: 2.7850, D(x): 3.04, D(G(z)): -2.79
Epoch [2/200], Step [600/600], d_loss: -5.2225, g_loss: 4.0367, D(x): 1.66, D(G(z)): -4.04
Epoch [3/200], Step [200/600], d_loss: -5.3969, g_loss: 3.0893, D(x): 2.97, D(G(z)): -3.09
Epoch [3/200], Step [400/600], d_loss: -5.0331, g_loss: 3.7611, D(x): 1.58, D(G(z)): -3.7

In [0]:
# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

In [14]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
cntr = 1
for blk in range(20):
 img_arr = [cv2.imread('/content/samples/fake_images-'+str(cntr)+'.png')]
 cntr += 1
 for i in range(1,10):
   img_arr.append(cv2.imread('/content/samples/fake_images-'+str(cntr)+'.png'))
   cntr += 1
 pctrs = np.hstack(img_arr)
 print("<<<<<<<<<>>>>>>>>>>")
 print("Batch :",blk)
 print("<<<<<<<<<>>>>>>>>>>")
 cv2_imshow(pctrs)

Output hidden; open in https://colab.research.google.com to view.

In [21]:
!zip -r /content/training_fake_images.zip /content/samples

  adding: content/samples/ (stored 0%)
  adding: content/samples/fake_images-56.png (deflated 5%)
  adding: content/samples/fake_images-25.png (deflated 5%)
  adding: content/samples/fake_images-35.png (deflated 5%)
  adding: content/samples/fake_images-8.png (deflated 4%)
  adding: content/samples/fake_images-195.png (deflated 5%)
  adding: content/samples/fake_images-29.png (deflated 5%)
  adding: content/samples/fake_images-137.png (deflated 5%)
  adding: content/samples/fake_images-108.png (deflated 5%)
  adding: content/samples/fake_images-1.png (deflated 3%)
  adding: content/samples/fake_images-17.png (deflated 5%)
  adding: content/samples/fake_images-127.png (deflated 5%)
  adding: content/samples/fake_images-117.png (deflated 4%)
  adding: content/samples/fake_images-182.png (deflated 5%)
  adding: content/samples/fake_images-100.png (deflated 5%)
  adding: content/samples/fake_images-157.png (deflated 5%)
  adding: content/samples/fake_images-7.png (deflated 4%)
  adding: co